In [ ]:
# Preprocessing dataset

In [ ]:
# step 1) Normalizing each 3d .nii file,  then convert all normalized 3d .nii volumes into  2d bmp slices, for both the Images and their corresponding Masks. 

import nibabel as nib
import cv2
import numpy as np
import math
import os

maxno=76  # number of patients in the dataset

def do_extract(nii_path, path_to_extract, mask):
    nii = nib.load(str(nii_path))
    nii = np.array(nii.dataobj)
    
    # nii.shape[2] is the number of slices
    dataset = [cv2.merge((nii[:,:,i],nii[:,:,i],nii[:,:,i])) for i in range(nii.shape[2])]
    if (mask):
        imgs = [cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX) for img in dataset]
    else:
        imgs=dataset
    
    for index, img in enumerate(imgs):
        image_array = np.array(img)
        normalized_data=image_array
        if ~(mask):
            threshold=300
            filtered_data =  np.where(image_array > threshold, threshold, image_array)
            max_value=  np.max(filtered_data)
            min_value=  np.min(filtered_data)
            
            b = math.log(max_value - min_value + 1)
            if b==0: 
                b=b+1
            a=255/b
            transformed_data= a*np.log(filtered_data-min_value+1)
            normalized_data = transformed_data
        file_name = str(index).zfill(3) + ".bmp"
        file_path = os.path.join(path_to_extract, file_name)
        
        #Write image to output folder
        cv2.imwrite(file_path, normalized_data)
        print("Extract Done", str(index).zfill(3))
    
if __name__ == "__main__":
    source_dir = "/path/to/dataset/3D" #3D dataset path
    dest_dir= "/path/to/dataset/2D/Pts/" #2D dataset path

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    for j in range(1, maxno):
        if j<10:
            i= "Pt 000"+str(j)
        else :
            i= "Pt 00"+str(j)
      
        nii_path_Images = source_dir+'/Images/' + i + '.nii'
        #print(nii_path_Images)

        nii_path_Masks = source_dir + '/Masks/'+ i + '_Segmentation.nii'
        #print(nii_path_Masks)

        if not os.path.exists(nii_path_Images):
            print ('the path was not foun for Images')
            continue
    
        if not os.path.exists(nii_path_Masks):
            print ('the path was not found for Masks')
            continue
    
        path_to_extract_Images = dest_dir + i + '/Images/'
        path_to_extract_Masks = dest_dir + i + '/Masks/'
        
        if not os.path.exists(path_to_extract_Images):
            os.makedirs(path_to_extract_Images)
        mask=0
        do_extract(nii_path_Images,path_to_extract_Images,mask)
    
        mask=1
        if not os.path.exists(path_to_extract_Masks):
            os.makedirs(path_to_extract_Masks)
        do_extract(nii_path_Masks,path_to_extract_Masks,mask)
        

In [ ]:
# Step2) Organizng the Images folder : adding each patient's name as a prefix to their corresponding image filenames and copy all updated files into the Images folder 
#                                      Repeat the same procedure, modifying only source(src) and destination(dest) paths to correspond to the Masks folder
from openpyxl import load_workbook
import re
import shutil
from pathlib import Path

dest='/path/to/dataset/2D/Pts/2D/All Images'  

if not os.path.exists(dest):
    os.makedirs(dest)

data_file = '/path/to/dataset/Patients_info.xlsx' # This Excell sheet provides, for each patient, the slice numbers marking the start and end of the  ablation zone

wb = load_workbook(data_file)
for sheetname in wb.sheetnames:
    print(sheetname)

ws = wb[sheetname]
all_rows = list(ws.rows)

maxno=76

for row in all_rows[1:maxno]:
    #patient_no coloumn 0
    patient_no = row[0].value
    print(f"{patient_no} patient number")

    #first slice for patient_no (coloumn 3)
    patient_firstslice = row[4].value
    #print(f" {patient_firstslice} first slice number")

    #last slice for patient_no (coloumn 4)
    patient_lastslice = row[5].value
    #print(f" {patient_lastslice} last slice number")

    j=int(patient_no)

    if j<10:
        i= "Pt 000"+str(j)
    else :
        i= "Pt 00"+str(j)
        
    src='/path/to/dataset/2D/Pts/2D/Pts/'+ i + '/Images/'

    src_files = os.listdir(src)

    for file_name in src_files:
        #print('file_name is', file_name)  
        file_name_no = re.search('(.+?).bmp', file_name)
        file_name_no = file_name_no.group(1)
            
        fn=int(file_name_no)
        
        if patient_firstslice <=fn  and fn <= patient_lastslice :
            full_file_name = os.path.join(src, file_name)
            if os.path.isfile(full_file_name):
                #print('full_file_name is',full_file_name)
                shutil.copy(full_file_name, dest)
                print ('Copy',full_file_name,'to ',dest,' done')
            
    root_dir = Path(dest)
    print('dest is', root_dir)
    file_paths = root_dir.iterdir()
    for path in file_paths:
        print('path is ', path)
        new_filename= i +'_'+ path.stem + path.suffix
        new_filepath=path.with_name(new_filename)
        print('new_filepath is ',new_filepath)
        os.rename(path, new_filename)
        print('..... rename done.....')